In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


In [14]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT ii.* FROM invoice_items ii
    INNER JOIN invoices i on ii.InvoiceId = i.InvoiceId
    INNER JOIN customers c on i.CustomerId = c.CustomerId
    WHERE c.country = "USA"
   )

SELECT * FROM usa_tracks_sold LIMIT 5;

 * sqlite:///chinook.db
Done.


InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
74,13,462,0.99,1
723,134,907,0.99,1
724,134,909,0.99,1
782,145,1236,0.99,1
783,145,1245,0.99,1


In [17]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT ii.* FROM invoice_items ii
    INNER JOIN invoices i on ii.InvoiceId = i.InvoiceId
    INNER JOIN customers c on i.CustomerId = c.CustomerId
    WHERE c.country = "USA"
   )

SELECT
    g.Name genres,
    count(uts.InvoiceLineId) tracks_sold,
    cast(count(uts.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN tracks t on t.TrackId = uts.TrackId
INNER JOIN genres g on g.GenreId = t.GenreId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genres,tracks_sold,percentage_sold
Rock,157,0.31781376518218624
Latin,91,0.18421052631578946
Metal,64,0.12955465587044535
Alternative & Punk,50,0.10121457489878542
Jazz,22,0.044534412955465584
Blues,15,0.030364372469635626
TV Shows,14,0.02834008097165992
R&B/Soul,12,0.024291497975708502
Classical,8,0.016194331983805668
Comedy,8,0.016194331983805668
